In [2]:
import cv2
import glob
import numpy as np
from skimage.feature import hog, local_binary_pattern
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns  # Para la visualización de la matriz de confusión

# Definir las carpetas de frutas frescas y podridas
freshfruits = {
    "freshapples": "freshapple",
    "freshbanana": "freshbanana",
    "freshoranges": "freshorange"
}

rottenfruits = {
    "rottenapples": "rottenapple",
    "rottenbanana": "rottenbanana",
    "rottenoranges": "rottenorange"
}

# Inicializar listas para almacenar características y etiquetas
data_train = []
labels_train = []
data_test = []
labels_test = []

# Parámetros para LBP
LBP_RADIUS = 1
LBP_N_POINTS = 8 * LBP_RADIUS
LBP_METHOD = 'uniform'

# Función para procesar imágenes y extraer características HOG, histogramas de color y LBP
def process_images(folder_dict, base_path, data_list, labels_list):
    for folder, label in folder_dict.items():
        folder_path = f"{base_path}/{folder}/*.png"
        for image_path in glob.glob(folder_path):
            img = cv2.imread(image_path)
            if img is None or len(img.shape) != 3 or img.shape[2] != 3:
                print(f"Imagen no válida o con canales incorrectos: {image_path}")
                continue
            try:
                # Redimensionar la imagen
                resized_img = cv2.resize(img, (64, 32))
                
                # Conversión a escala de grises para HOG y LBP
                gray_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)
                
                # Extracción de características HOG
                hog_features = hog(
                    gray_img,
                    orientations=9,
                    pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2),
                    visualize=False,
                    feature_vector=True,
                    block_norm='L2-Hys'
                )
                
                # Extracción de histogramas de color
                hist_features = []
                for i in range(3):  # Canales B, G, R
                    hist = cv2.calcHist([resized_img], [i], None, [256], [0, 256])
                    hist = cv2.normalize(hist, hist).flatten()
                    hist_features.extend(hist)
                
                # Extracción de características LBP
                lbp = local_binary_pattern(gray_img, LBP_N_POINTS, LBP_RADIUS, LBP_METHOD)
                lbp_hist, _ = np.histogram(lbp.ravel(),
                                           bins=np.arange(0, LBP_N_POINTS + 3),
                                           range=(0, LBP_N_POINTS + 2))
                lbp_hist = lbp_hist.astype("float")
                lbp_hist /= (lbp_hist.sum() + 1e-6)  # Normalizar el histograma
                lbp_features = lbp_hist
                
                # Concatenar características HOG, histogramas de color y LBP
                features = np.concatenate([hog_features, hist_features, lbp_features])
            except Exception as e:
                print(f"Error procesando {image_path}: {e}")
                continue

            # Agregar las características y la etiqueta a las listas
            data_list.append(features)
            labels_list.append(label)

# Procesar imágenes de entrenamiento
train_base_path = ".gitignore/dataset/train"
process_images(freshfruits, train_base_path, data_train, labels_train)
process_images(rottenfruits, train_base_path, data_train, labels_train)

print("Total de imágenes de entrenamiento procesadas:", len(data_train))

# Procesar imágenes de prueba
test_base_path = ".gitignore/dataset/test"
process_images(freshfruits, test_base_path, data_test, labels_test)
process_images(rottenfruits, test_base_path, data_test, labels_test)

print("Total de imágenes de test procesadas:", len(data_test))

# Convertir las características y etiquetas a arrays de NumPy
X_train = np.array(data_train)
y_train_labels = np.array(labels_train)
X_test = np.array(data_test)
y_test_labels = np.array(labels_test)

# Codificar las etiquetas de forma consistente
encoder = LabelEncoder()
encoder.fit(np.concatenate((y_train_labels, y_test_labels)))

y_train = encoder.transform(y_train_labels)
y_test = encoder.transform(y_test_labels)

Total de imágenes de entrenamiento procesadas: 10901
Total de imágenes de test procesadas: 2698


In [3]:
# SVM amb Kernel Polinòmic - Model de Classificació
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Definir i entrenar el model SVM amb kernel polinòmic
poly_classifier = SVC(kernel='poly', degree=3, coef0=1, probability=True, random_state=42)
poly_classifier.fit(X_train, y_train)

# Predir sobre el conjunt de prova
y_pred_poly = poly_classifier.predict(X_test)

# Avaluar el model
accuracy_poly = accuracy_score(y_test, y_pred_poly)
print("----- SVM amb Kernel Polinòmic -----")
print(f"Accuracy: {accuracy_poly:.4f}\n")
print("Informe de classificació:")
print(classification_report(y_test, y_pred_poly, target_names=encoder.classes_))
print("Matriu de Confusió:")
print(confusion_matrix(y_test, y_pred_poly))

----- SVM amb Kernel Polinòmic -----
Accuracy: 0.9433

Informe de classificació:
              precision    recall  f1-score   support

  freshapple       0.92      0.92      0.92       395
 freshbanana       0.97      0.97      0.97       381
 freshorange       0.98      0.97      0.97       388
 rottenapple       0.90      0.93      0.91       601
rottenbanana       0.98      0.98      0.98       530
rottenorange       0.92      0.89      0.91       403

    accuracy                           0.94      2698
   macro avg       0.95      0.94      0.94      2698
weighted avg       0.94      0.94      0.94      2698

Matriu de Confusió:
[[365   1   0  26   0   3]
 [  0 369   0   1   9   2]
 [  2   0 376   3   0   7]
 [ 23   1   2 556   1  18]
 [  1   8   0   1 520   0]
 [  4   0   7  31   2 359]]
